In [1]:
import sys
if './' not in sys.path:
	sys.path.append('./')
	
from omegaconf import OmegaConf
import argparse

from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from ldm.util import instantiate_from_config
from models.util import load_state_dict
from models.logger import ImageLogger
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [14]:
config_path = './configs/video_uni_v15/local_v15-seg.yaml'
# config_path = './configs/local_v15.yaml'
config = OmegaConf.load(config_path)
# model = instantiate_from_config(config['model'])
# if args.resume_path:
#     model.load_state_dict(load_state_dict(args.resume_path, location='cpu'))
# model.learning_rate = learning_rate
# model.sd_locked = sd_locked

dataset = instantiate_from_config(config['data'])
dataloader = DataLoader(dataset, num_workers=4, batch_size=4, pin_memory=True, shuffle=True)

In [16]:
# config_path = './configs/local_v15-seg.yaml'
learning_rate = 1e-5
batch_size = 4
training_steps = 1e5
resume_path = './ckpt/init_local.ckpt'
default_logdir = './log_local/'
logger_freq = 500
sd_locked = True
num_workers = 4
gpus = -1

In [17]:
config = OmegaConf.load(config_path)
dataset = instantiate_from_config(config['data'])
dataloader = DataLoader(dataset, num_workers=num_workers, batch_size=batch_size, pin_memory=True, shuffle=True)

In [12]:
print(dataset[1]['jpg'].shape, dataset[1]['local_conditions'].shape)

(512, 512, 3) (512, 512, 21)


In [18]:
model = instantiate_from_config(config['model'])
# model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpkz9c0j38
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpkz9c0j38/_remote_module_non_scriptable.py


UniControlNet: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, conte

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.3.self_attn.q_proj.bias', 'vision_model.encoder.layers.13.self_attn.v_proj.weight', 'vision_model.encoder.layers.0.self_attn.v_proj.weight', 'vision_model.encoder.layers.13.layer_norm2.weight', 'vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.18.self_attn.k_proj.bias', 'vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_model.encoder.layers.3.layer_norm2.bias', 'vision_model.encoder.layers.4.mlp.fc2.weight', 'vision_model.encoder.layers.7.mlp.fc2.weight', 'vision_model.encoder.layers.19.self_attn.q_proj.weight', 'vision_model.encoder.layers.21.self_attn.v_proj.weight', 'vision_model.encoder.layers.7.self_attn.q_proj.bias', 'vision_model.encoder.layers.9.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.layer_norm2.weight', 'vision_model.encoder.layers.8.mlp.fc1.weight', 'vision_model.encoder.layers.19.

Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is 768 and using 8 heads.
Setting up Me

In [19]:
for batch in dataloader:
    print(batch['local_conditions'][0].shape)

torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])
torch.Size([512, 512, 3])


In [20]:
logger = ImageLogger(batch_frequency=logger_freq)
checkpoint_callback = ModelCheckpoint(every_n_train_steps=logger_freq,)

In [23]:
trainer = pl.Trainer(
        gpus=gpus,
        callbacks=[checkpoint_callback], 
        default_root_dir=default_logdir,
        max_steps=training_steps,
        strategy='dp'
    )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [24]:
trainer.fit(model,dataloader)

/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:326: LightningDeprecationWarning: Base `LightningModule.on_train_batch_start` hook signature has changed in v1.5. The `dataloader_idx` argument will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | model             | DiffusionWrapper   | 859 M 
1 | first_stage_model | AutoencoderKL      | 83.7 M
2 | cond_stage_model  | FrozenCLIPEmbedder | 123 M 
3 | local_adapter     | LocalAdapter       | 41

Training: 0it [00:00, ?it/s]

/data/maryam.sana/anaconda3/envs/unicontrol/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
